In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.models import Model, Sequential

import pandas as pd

In [2]:
data_path = 'data/cifar-10/'
train_path = data_path + 'trainLabels.csv'

# dfs contain filenames under "id" 
# we need to indicate we want our to load this in as string data
train_df = pd.read_csv(train_path, dtype='str')

In [3]:
train_df['label'].value_counts()

cat           5000
frog          5000
horse         5000
airplane      5000
bird          5000
automobile    5000
ship          5000
deer          5000
truck         5000
dog           5000
Name: label, dtype: int64

In [4]:
def append_ext(filename):
    '''
    appends 'png' file extension to a filename (str)
    '''
    return filename+'.png'

In [5]:
train_df['id'] = train_df['id'].apply(append_ext)

In [6]:
train_df.head()

,id,label
0,1.png,frog
1,2.png,truck
2,3.png,truck
3,4.png,deer
4,5.png,automobile


In [7]:
classes = list(train_df.label.unique())

datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.2)

train_generator=datagen.flow_from_dataframe(dataframe=train_df,
                                            directory=(data_path+"train/"),
                                            x_col="id",
                                            y_col="label",
                                            subset="training",
                                            classes=classes,
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(32,32))

valid_generator=datagen.flow_from_dataframe(dataframe=train_df,
                                            directory=(data_path+"train/"),
                                            x_col="id",
                                            y_col="label",
                                            subset="validation",
                                            classes=classes,
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(32,32))

Found 40000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


In [8]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

model.compile(optimizers.RMSprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [9]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model.fit(train_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=valid_generator,
          validation_steps=STEP_SIZE_VALID,
          epochs=10)

Epoch 1/10
1250/1250 [==============================] - 207s 165ms/step - loss: 1.8862 - accuracy: 0.3099 - val_loss: 1.5980 - val_accuracy: 0.4215
Epoch 2/10
1250/1250 [==============================] - 218s 175ms/step - loss: 1.5816 - accuracy: 0.4254 - val_loss: 1.4153 - val_accuracy: 0.4933
Epoch 3/10
1250/1250 [==============================] - 219s 175ms/step - loss: 1.4488 - accuracy: 0.4757 - val_loss: 1.3427 - val_accuracy: 0.5202
Epoch 4/10
1250/1250 [==============================] - 136s 108ms/step - loss: 1.3561 - accuracy: 0.5130 - val_loss: 1.2453 - val_accuracy: 0.5483
Epoch 5/10
1250/1250 [==============================] - 153s 122ms/step - loss: 1.2864 - accuracy: 0.5429 - val_loss: 1.1710 - val_accuracy: 0.5838
Epoch 6/10
1250/1250 [==============================] - 221s 177ms/step - loss: 1.2229 - accuracy: 0.5672 - val_loss: 1.1356 - val_accuracy: 0.5942
Epoch 7/10
1250/1250 [==============================] - 222s 177ms/step - loss: 1.1635 - accuracy: 0.5900 - val_